In [1]:
from pynq import Overlay, DefaultIP 

import struct

overlay = Overlay('/home/xilinx/pynq/overlays/quadratic_2/quad_func.bit')

add_ip = overlay.quadratic

class QuadDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)

    bindto = ['xilinx.com:hls:add:1.0']

    def quadratic(self, a, b, c, d, e, f, g, h, i, j, px, py, pz, dx, dy, dz, test1):
        add_ip.write(0x10, struct.unpack('I', struct.pack('f', a))[0])
        add_ip.write(0x18, struct.unpack('I', struct.pack('f', b))[0])
        add_ip.write(0x20, struct.unpack('I', struct.pack('f', c))[0])
        add_ip.write(0x28, struct.unpack('I', struct.pack('f', d))[0])
        add_ip.write(0x30, struct.unpack('I', struct.pack('f', e))[0])
        add_ip.write(0x38, struct.unpack('I', struct.pack('f', f))[0])
        add_ip.write(0x40, struct.unpack('I', struct.pack('f', g))[0])
        add_ip.write(0x48, struct.unpack('I', struct.pack('f', h))[0])
        add_ip.write(0x50, struct.unpack('I', struct.pack('f', i))[0])
        add_ip.write(0x58, struct.unpack('I', struct.pack('f', j))[0])
        add_ip.write(0x60, struct.unpack('I', struct.pack('f', px))[0])
        add_ip.write(0x68, struct.unpack('I', struct.pack('f', py))[0])
        add_ip.write(0x70, struct.unpack('I', struct.pack('f', pz))[0])
        add_ip.write(0x78, struct.unpack('I', struct.pack('f', dx))[0])
        add_ip.write(0x80, struct.unpack('I', struct.pack('f', dy))[0])
        add_ip.write(0x88, struct.unpack('I', struct.pack('f', dz))[0])
        add_ip.write(0x90, struct.unpack('I', struct.pack('f', test1))[0])
        
        test2 = struct.unpack('f', struct.pack('I', add_ip.read(0x98)))[0]

        hit = bool(add_ip.read(0xa8))
        t = struct.unpack('f', struct.pack('I', add_ip.read(0xb8)))[0]
        pox = struct.unpack('f', struct.pack('I', add_ip.read(0xc8)))[0]
        poy = struct.unpack('f', struct.pack('I', add_ip.read(0xd8)))[0]
        poz = struct.unpack('f', struct.pack('I', add_ip.read(0xe8)))[0]
        nox = struct.unpack('f', struct.pack('I', add_ip.read(0xf8)))[0]
        noy = struct.unpack('f', struct.pack('I', add_ip.read(0x108)))[0]
        noz = struct.unpack('f', struct.pack('I', add_ip.read(0x118)))[0]

        return (test2, hit, t, pox, poy, poz, nox, noy, noz)

overlay = Overlay('/home/xilinx/pynq/overlays/quadratic_2/quad_func.bit')


ModuleNotFoundError: No module named 'pynq'

In [2]:

from pynq import Overlay
import struct
from pynq import allocate
import numpy as np 

overlay2 = Overlay('/home/xilinx/pynq/overlays/updated_multiple/dma_master_vivado.bit')

dma_ip = overlay2.dma_master_test_0

dma_ip.register_map

ModuleNotFoundError: No module named 'pynq'

In [3]:
from structures.hit import Hit

def driver_dma(quad_arr, ray_arr, obj):
    in_buffer[:] = np.concatenate((np.zeros(shape=(7,), dtype="float32"), ray_arr, quad_arr))
    dma_ip.register_map.num_quads = 1

    dma_ip.register_map.CTRL.AP_START=1
    #while (dma_ip.register_map.CTRL.AP_DONE == 0):
    #    continue

    temp_hit = False
    if (in_buffer[0] != 0):
        temp_hit = True
    return Hit(temp_hit, in_buffer[0], obj, Vertex(in_buffer[1], in_buffer[2], in_buffer[3]), Vector(in_buffer[4], in_buffer[5], in_buffer[6]))

max_objs = 50

def setup_buffers(quad_arrs, num_objs):
    curr_objs = 0
    buffer_array = []
    while (curr_objs < num_objs):
        top = min(curr_objs+max_objs, num_objs)
        in_buffer = allocate(shape=(((top - curr_objs) * 10) + 14,), dtype="float32")
        in_buffer[14:] = quad_arrs[curr_objs*10:top*10]
        buffer_array.append(in_buffer)
        curr_objs += (top - curr_objs)
        
    #dma_ip.register_map.arr_1 = in_buffer.physical_address
    #in_buffer[14:] = quad_arrs
    #dma_ip.register_map.num_quads = num_objs
    return buffer_array



def driver_multiple_dma(in_buffer, ray_arr):
    smallest_hit = Hit(False, None, None, None, None)
    out_i = 0
    for i in range(len(in_buffer)):
        arr = in_buffer[i]
        arr[8:14] = ray_arr
        dma_ip.register_map.arr_1 = arr.physical_address
        dma_ip.register_map.num_quads = int((len(arr) - 14) / 10)
        dma_ip.register_map.CTRL.AP_START=1
        while (dma_ip.register_map.CTRL.AP_DONE == 0):
            continue
        
        if (arr[0] != -1 and (not smallest_hit.flag or smallest_hit.t > arr[0])):
            smallest_hit = Hit(True, arr[0], int(arr[7]), Vertex(arr[1], arr[2], arr[3]), Vector(arr[4], arr[5], arr[6]))
            out_i = i
    return (smallest_hit, max_objs, out_i)


"""
def setup_buffers(quad_arrs, num_objs):
    curr_objs = 0
    buffer_array = []
    while (curr_objs < num_objs):
        
        in_buffer = allocate(shape=((num_objs * 10) + 14,), dtype="float32")
    dma_ip.register_map.arr_1 = in_buffer.physical_address
    in_buffer[14:] = quad_arrs
    dma_ip.register_map.num_quads = num_objs
    return in_buffer

"""

'\ndef setup_buffers(quad_arrs, num_objs):\n    curr_objs = 0\n    buffer_array = []\n    while (curr_objs < num_objs):\n        \n        in_buffer = allocate(shape=((num_objs * 10) + 14,), dtype="float32")\n    dma_ip.register_map.arr_1 = in_buffer.physical_address\n    in_buffer[14:] = quad_arrs\n    dma_ip.register_map.num_quads = num_objs\n    return in_buffer\n\n'

In [4]:
from buffer import Buffer

from structures.material import Material
from structures.ray import Ray

from structures.scene import Scene

from sphere import Sphere
from quadratic import Quadratic
from structures.light import Light
from structures.pixel import Pixel
from structures.vertex import Vertex
from structures.vector import Vector
import time


driver = None
#driver = overlay.quadratic.quadratic
#driver = driver_multiple_dma
print(driver)

def main():
    timings = []
    #listing = [1, 49, 50, 51, 99, 100]
    for i in range(1, 501):
        timings.append(standard(i))
        
    print(timings)
    #multiple()
    # test = Quadratic(1, 0, 0, 2, 1, 0, 0, 1, 0, -1, Material(Pixel(0.1, 0.1, 0.1), Pixel(0.5, 0, 0), Pixel(0.5, 0.5, 0.5), 50), driver)
    # x = test.intersection(Ray(Vertex(0, 0, 0), Vector(-0.3015113, -0.3015113, 0.9045340)))
    # print(x)
    # print((x.flag, x.t, x.position.x, x.position.y, x.position.z, x.normal.x, x.normal.y, x.normal.z))


None


In [5]:
def standard(num_object_test):
    size = 10
    half_size = size / 2
    half_size_i = int(half_size)
    eye = Vertex(0, 0, -5)
    buff = Buffer(size, size)

    # sph = Sphere(Vertex(0, 0, 0), 1, Material(Pixel(0.1, 0.1, 0.1), Pixel(0.3, 0.3, 0.3), Pixel(0.5, 0.5, 0.5), 50))
    # sph2 = Sphere(Vertex(0, 0, 0), 1, Material(Pixel(0.1, 0.1, 0.1), Pixel(0.3, 0.3, 0.3), Pixel(0.5, 0.5, 0.5), 50))
    test = Quadratic(1, 0, 0, 0, 1, 0, 0, 1, 0, -1, Material(Pixel(0.1, 0.1, 0.1), Pixel(0.5, 0, 0), Pixel(0.5, 0.5, 0.5), 50))
    test2 = Quadratic(1, 0, 0, -1, 1, 0, 0, 1, 0, -1, Material(Pixel(0.1, 0.1, 0.1), Pixel(0, 0.6, 0), Pixel(0.5, 0.5, 0.5), 50))
    #objs = [test, test2, test3, test4, test5, test6, test7, test8]
    #objs = [test, test2, test3, test4, test5, test6, test7, test8, test9, test10, test11, test12, test13, test14, test15, test16, 
    #        test, test2, test3, test4, test5, test6, test7, test8, test9, test10, test11, test12, test13, test14, test15, test16]
    
    print(num_object_test)
    
    #objs = [test, test, test, test2, test2]
    objs = []
    for i in range(num_object_test):
        objs.append(test)
        
    light1 = Light(Vertex(4, 2, -5), Pixel(1, 1, 1), 1, 1, 0)

    lights = [light1]

    total_time = time.time()
    
    scene_time = 0
    scene_pass = 0
    
    quad_time = 0
    quad_pass = 0
    quad_iter = len(objs)
    
    scene = Scene(eye, lights, objs, None) #setup_buffers
    
    for x in range(-half_size_i, half_size_i):
        #print(x)
        for y in range(-half_size_i, half_size_i):
            ray = Ray(eye, Vector(x / half_size, y / half_size, 3).normalise())
            s_t_s = time.time()
            render_res = scene.render(ray, driver)
            s_t_e = time.time()
            scene_time += (s_t_e - s_t_s)
            scene_pass += 1
            quad_time += render_res[2]
            quad_pass += quad_iter
            if (render_res[0] == False):
                continue
            buff.plot_pixel(x + half_size_i, y + half_size_i, render_res[1])


    total_time = (time.time() - total_time)
    buff.write_file("test_multiple_scene_2.png")
    return { len(objs): [quad_time / quad_pass, scene_time / scene_pass, total_time] }

    print("Time per quad: ", quad_time / quad_pass)
    print("Time per scene: ", scene_time / scene_pass)
    print("Total time: ", total_time)
    
    print(quad_pass)
    
    print("Writing file.")


In [6]:
main()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
[(0.0005101798772811889, 0.0010754912853240966, 13.305267810821533), (0.0002548795461654663, 0.0010762730836868287, 13.313583374023438), (0.0001704332192738851, 0.001079407572746277, 13.350334882736206)]

In [ ]:
DMA:
    
[{1: [0.0005431119680404663, 0.0017272896528244019, 19.806490421295166]}, {2: [0.0002698556184768677, 0.001722495484352112, 19.77220630645752]}, {3: [0.0001810603141784668, 0.001727178645133972, 19.82813596725464]}, {4: [0.00013551220297813415, 0.0017215410232543945, 19.753669500350952]}, {5: [0.00010793148517608642, 0.001726850962638855, 19.8258855342865]}, {6: [9.10197655359904e-05, 0.001735815691947937, 19.919672966003418]}, {7: [7.758755343300956e-05, 0.0017281980752944946, 19.830491542816162]}, {8: [6.753036379814149e-05, 0.0017258009433746338, 19.80319046974182]}, {9: [6.026891867319743e-05, 0.0017300621271133423, 19.862537384033203]}, {10: [5.450664281845093e-05, 0.0017349540233612061, 19.9058837890625]}, {11: [4.9420376257462935e-05, 0.0017252469062805176, 19.809868097305298]}, {12: [4.503917892773946e-05, 0.0017219481945037842, 19.77206540107727]}, {13: [4.187616751744197e-05, 0.0017300145149230957, 19.86135196685791]}, {14: [3.815037863595145e-05, 0.0017150130271911621, 19.688904285430908]}, {15: [3.6314590771993e-05, 0.0017304446935653686, 19.861334323883057]}, {16: [3.393288999795914e-05, 0.0017234033823013307, 19.78608798980713]}, {17: [3.193377046024098e-05, 0.0017226156711578368, 19.777466773986816]}, {18: [3.0058340231577557e-05, 0.0017224098920822144, 19.779162883758545]}, {19: [2.8508570319727847e-05, 0.0017244103670120239, 19.795802354812622]}, {20: [2.679091691970825e-05, 0.001714214038848877, 19.68684220314026]}, {21: [2.5632561956133162e-05, 0.0017170793533325195, 19.72691035270691]}, {22: [2.46144099669023e-05, 0.0017214212179183959, 19.765655040740967]}, {23: [2.3558067238849143e-05, 0.0017205344200134278, 19.759875059127808]}, {24: [2.2568599383036295e-05, 0.0017265979766845703, 19.82269787788391]}, {25: [2.1731887817382812e-05, 0.0017289679288864136, 19.84402060508728]}, {26: [2.090985683294443e-05, 0.0017259955167770386, 19.811599016189575]}, {27: [2.0166345878883646e-05, 0.0017314812421798706, 19.871214866638184]}, {28: [1.9454034737178256e-05, 0.0017274068355560303, 19.839099645614624]}, {29: [1.8726232134062667e-05, 0.0017202546358108522, 19.756725549697876]}, {30: [1.8082760175069174e-05, 0.001723044729232788, 19.78150486946106]}, {31: [1.7585547508731964e-05, 0.001727102303504944, 19.837639808654785]}, {32: [1.7019572108983994e-05, 0.001727055859565735, 19.83343482017517]}, {33: [1.651659734321363e-05, 0.0017281437158584594, 19.83605694770813]}, {34: [1.5965551488539752e-05, 0.0017200244188308716, 19.754775285720825]}, {35: [1.555123669760568e-05, 0.001726853919029236, 19.834288597106934]}, {36: [1.4870049556096395e-05, 0.001715140151977539, 19.7087345123291]}, {37: [1.4610862087559055e-05, 0.0017190866947174072, 19.750329971313477]}, {38: [1.4236770178142347e-05, 0.0017185728073120118, 19.745200395584106]}, {39: [1.393128786331568e-05, 0.001721293568611145, 19.77251100540161]}, {40: [1.3489652276039124e-05, 0.0017164911031723022, 19.715519189834595]}, {41: [1.3190115370401521e-05, 0.0017189139127731323, 19.744385480880737]}, {42: [1.2768525736672538e-05, 0.001715779709815979, 19.712204694747925]}, {43: [1.2568892434585926e-05, 0.0017178441047668456, 19.73032236099243]}, {44: [1.2325391444292936e-05, 0.001721012854576111, 19.774561405181885]}, {45: [1.2072427537706163e-05, 0.0017235049962997437, 19.79491114616394]}, {46: [1.1748995988265327e-05, 0.0017199414968490602, 19.748520612716675]}, {47: [1.1526170182735362e-05, 0.0017209553480148315, 19.76400327682495]}, {48: [1.121626595656077e-05, 0.0017199318408966065, 19.755115032196045]}, {49: [1.1042904367252273e-05, 0.0017230783462524415, 19.793967485427856]}]
Python:
    
[{1: [0.0002387103796005249, 0.0006598797082901001, 9.01147723197937]}, {2: [0.00022365880012512206, 0.0008685851335525512, 11.106118440628052]}, {3: [0.00021801133950551352, 0.001076094388961792, 13.180331945419312]}, {4: [0.00021392916440963745, 0.001280285859107971, 15.225905418395996]}, {5: [0.00021358229637145996, 0.0014958619832992554, 17.397912979125977]}, {6: [0.00021005425055821737, 0.0016868484735488892, 19.302841186523438]}, {7: [0.0002098095144544329, 0.0018964375734329224, 21.40614414215088]}, {8: [0.00020817059874534606, 0.0020918861865997316, 23.365346431732178]}, {9: [0.00020755981869167753, 0.002295800256729126, 25.400399208068848]}, {10: [0.00020787601232528685, 0.0025081286907196044, 27.52853226661682]}, {11: [0.00020659196376800537, 0.0027019320249557495, 29.47439193725586]}, {12: [0.0002064779043197632, 0.0029076266527175904, 31.527666568756104]}, {13: [0.00020629437703352708, 0.003112311339378357, 33.57447385787964]}, {14: [0.00020751103503363472, 0.0033370547294616698, 35.84706473350525]}, {15: [0.00020586173216501872, 0.003518932485580444, 37.64771866798401]}, {16: [0.0002055614709854126, 0.003720987153053284, 39.67592000961304]}, {17: [0.00020642429520102107, 0.003942117643356323, 41.89858031272888]}, {18: [0.00020541369120279947, 0.004128691267967224, 43.75350546836853]}, {19: [0.00020591482112282202, 0.004346090221405029, 45.94392395019531]}, {20: [0.00020703518867492676, 0.0045753974199295045, 48.24669003486633]}, {21: [0.00020504196144285655, 0.004737956619262696, 49.85789394378662]}, {22: [0.00020638959407806397, 0.0049738689184188846, 52.219677448272705]}, {23: [0.0002073906784472258, 0.005205669355392456, 54.550662994384766]}, {24: [0.00020780799190203348, 0.005424612379074096, 56.741135597229004]}, {25: [0.00020449754524230957, 0.005546933150291443, 57.95632982254028]}, {26: [0.00020581420751718374, 0.005786867833137512, 60.349655628204346]}, {27: [0.0002051038212246365, 0.0059731365919113155, 62.22633767127991]}, {28: [0.00020478683369500297, 0.0061702244281768796, 64.20467233657837]}, {29: [0.00020486341920392267, 0.006377567172050476, 66.27745842933655]}, {30: [0.00020459582408269245, 0.006572646617889404, 68.21390962600708]}, {31: [0.00020508840237894367, 0.006793377804756164, 70.43350791931152]}, {32: [0.00020573686212301253, 0.007019694185256958, 72.69663190841675]}, {33: [0.00020502450321659897, 0.007201675748825073, 74.51683354377747]}, {34: [0.00020525555400287404, 0.007416094732284546, 76.6629273891449]}, {35: [0.0002054001542500087, 0.007625132298469543, 78.74842190742493]}, {36: [0.00020596563617388406, 0.007852475428581238, 81.03686547279358]}, {37: [0.00020517878596847122, 0.008028169322013855, 82.78695869445801]}, {38: [0.00020599375774985865, 0.008265452909469605, 85.16409420967102]}, {39: [0.0002064468169823671, 0.008491739726066589, 87.43205547332764]}, {40: [0.00020696172058582305, 0.008717565441131591, 89.69178247451782]}, {41: [0.00020623892225870273, 0.008894619989395142, 91.46094846725464]}, {42: [0.00020699531975246612, 0.009133046078681945, 93.85449147224426]}, {43: [0.00020700361506883488, 0.009341829657554626, 95.93082332611084]}, {44: [0.00020719891353086992, 0.009556361174583436, 98.0767548084259]}, {45: [0.00020618683444129096, 0.00971778666973114, 99.69323825836182]}, {46: [0.00020666887967482858, 0.009948142671585083, 102.00173544883728]}, {47: [0.0002072408802965854, 0.010182723689079284, 104.35649681091309]}, {48: [0.00020748129536708195, 0.010400817012786865, 106.53537702560425]}, {49: [0.00020685397800134154, 0.010578639769554138, 108.3187608718872]}]

In [ ]:
# 1 - 49
dma_data = np.array([[0.0005431119680404663,0.0017272896528244019,19.806490421295166],
[0.0002698556184768677,0.001722495484352112,19.77220630645752],
[0.0001810603141784668,0.001727178645133972,19.82813596725464],
[0.00013551220297813415,0.0017215410232543945,19.753669500350952],
[0.00010793148517608642,0.001726850962638855,19.8258855342865],
[0.0000910197655359904,0.001735815691947937,19.919672966003418],
[0.00007758755343300956,0.0017281980752944946,19.830491542816162],
[0.00006753036379814149,0.0017258009433746338,19.80319046974182],
[0.00006026891867319743,0.0017300621271133423,19.862537384033203],
[0.00005450664281845093,0.0017349540233612061,19.9058837890625],
[0.000049420376257462935,0.0017252469062805176,19.809868097305298],
[0.00004503917892773946,0.0017219481945037842,19.77206540107727],
[0.00004187616751744197,0.0017300145149230957,19.86135196685791],
[0.00003815037863595145,0.0017150130271911621,19.688904285430908],
[0.000036314590771993,0.0017304446935653686,19.861334323883057],
[0.00003393288999795914,0.0017234033823013307,19.78608798980713],
[0.00003193377046024098,0.0017226156711578368,19.777466773986816],
[0.000030058340231577557,0.0017224098920822144,19.779162883758545],
[0.000028508570319727847,0.0017244103670120239,19.795802354812622],
[0.00002679091691970825,0.001714214038848877,19.68684220314026],
[0.000025632561956133162,0.0017170793533325195,19.72691035270691],
[0.0000246144099669023,0.0017214212179183959,19.765655040740967],
[0.000023558067238849143,0.0017205344200134278,19.759875059127808],
[0.000022568599383036295,0.0017265979766845703,19.82269787788391],
[0.000021731887817382812,0.0017289679288864136,19.84402060508728],
[0.00002090985683294443,0.0017259955167770386,19.811599016189575],
[0.000020166345878883646,0.0017314812421798706,19.871214866638184],
[0.000019454034737178256,0.0017274068355560303,19.839099645614624],
[0.000018726232134062667,0.0017202546358108522,19.756725549697876],
[0.000018082760175069174,0.001723044729232788,19.78150486946106],
[0.000017585547508731964,0.001727102303504944,19.837639808654785],
[0.000017019572108983994,0.001727055859565735,19.83343482017517],
[0.00001651659734321363,0.0017281437158584594,19.83605694770813],
[0.000015965551488539752,0.0017200244188308716,19.754775285720825],
[0.00001555123669760568,0.001726853919029236,19.834288597106934],
[0.000014870049556096395,0.001715140151977539,19.7087345123291],
[0.000014610862087559055,0.0017190866947174072,19.750329971313477],
[0.000014236770178142347,0.0017185728073120118,19.745200395584106],
[0.00001393128786331568,0.001721293568611145,19.77251100540161],
[0.000013489652276039124,0.0017164911031723022,19.715519189834595],
[0.000013190115370401521,0.0017189139127731323,19.744385480880737],
[0.000012768525736672538,0.001715779709815979,19.712204694747925],
[0.000012568892434585926,0.0017178441047668456,19.73032236099243],
[0.000012325391444292936,0.001721012854576111,19.774561405181885],
[0.000012072427537706163,0.0017235049962997437,19.79491114616394],
[0.000011748995988265327,0.0017199414968490602,19.748520612716675],
[0.000011526170182735362,0.0017209553480148315,19.76400327682495],
[0.00001121626595656077,0.0017199318408966065,19.755115032196045],
[0.000011042904367252273,0.0017230783462524415,19.793967485427856]])


dma_data = np.array([
    [0.0011441469192504882, 0.0014878511428833008, 0.1752171516418457],
[0.0005478668212890625, 0.0014373016357421874, 0.17075037956237793],
[0.0003703896204630534, 0.0014519286155700683, 0.1710042953491211],
[0.0002744960784912109, 0.0014420723915100097, 0.17034244537353516],
[0.00021903467178344726, 0.0014377903938293456, 0.17038559913635254],
[0.00018774509429931641, 0.0014713573455810546, 0.173447847366333],
[0.00015716450554983957, 0.0014426755905151368, 0.17155885696411133],
[0.00013683080673217773, 0.001436135768890381, 0.17007040977478027],
[0.00012293656667073568, 0.001449601650238037, 0.17145657539367676],
[0.00010947299003601074, 0.0014386487007141114, 0.1706700325012207],
[0.00010002309625799005, 0.0014444780349731445, 0.17208504676818848],
[9.29868221282959e-05, 0.0014589667320251466, 0.17277741432189941],
[8.430590996375451e-05, 0.0014371562004089355, 0.17061805725097656],
[7.80127729688372e-05, 0.001434786319732666, 0.1719958782196045],
[7.378673553466796e-05, 0.0014488863945007324, 0.17189645767211914],
[6.872937083244324e-05, 0.0014414644241333007, 0.17232346534729004],
[6.438157137702494e-05, 0.0014404940605163574, 0.17278552055358887],
[6.167822413974337e-05, 0.0014568448066711425, 0.17308735847473145],
[5.782654410914371e-05, 0.001442596912384033, 0.17228269577026367],
[5.478274822235107e-05, 0.0014400076866149902, 0.17257356643676758],
[5.276475633893694e-05, 0.001450190544128418, 0.1723160743713379],
[4.9919756976040926e-05, 0.0014406132698059082, 0.172271728515625],
[4.7856724780538805e-05, 0.0014454436302185059, 0.1724252700805664],
[4.622320334116618e-05, 0.0014520788192749024, 0.17259716987609863],
[4.2747116088867187e-05, 0.001408698558807373, 0.16987133026123047],
[4.2997140150803786e-05, 0.0014598536491394042, 0.1755537986755371],
[4.108879301283095e-05, 0.0014519643783569335, 0.1727912425994873],
[3.8470285279410225e-05, 0.0014165663719177247, 0.17095279693603516],
[3.803639576352876e-05, 0.001445181369781494, 0.1743767261505127],
[3.6888758341471355e-05, 0.0014481234550476074, 0.1727132797241211],
[3.537601040255639e-05, 0.001437058448791504, 0.17313718795776367],
[3.421634435653686e-05, 0.0014366745948791505, 0.17235803604125977],
[3.3517028346206206e-05, 0.0014468026161193847, 0.172987699508667],
[3.1324344522812785e-05, 0.0014035892486572265, 0.17008733749389648],
[3.202956063406808e-05, 0.0014608597755432129, 0.17671680450439453],
[3.073248598310683e-05, 0.001446235179901123, 0.17332196235656738],
[2.8824677338471285e-05, 0.0014044022560119628, 0.1704409122467041],
[2.8844318891826428e-05, 0.0014346194267272948, 0.17406177520751953],
[2.8396202967717097e-05, 0.0014482712745666503, 0.17374277114868164],
[2.6765227317810058e-05, 0.0014086222648620605, 0.17110228538513184],
[2.672428038062119e-05, 0.0014339113235473633, 0.17428207397460938],
[2.6177565256754557e-05, 0.001441657543182373, 0.1733074188232422],
[2.4763373441474382e-05, 0.0014027833938598632, 0.17026400566101074],
[2.4788867343555797e-05, 0.001432480812072754, 0.1744403839111328],
[2.4427784813774957e-05, 0.0014412617683410645, 0.17365694046020508],
[2.3537511410920517e-05, 0.001422436237335205, 0.17348837852478027],
[2.329892300544901e-05, 0.0014345622062683105, 0.175034761428833],
[2.3046682278315228e-05, 0.0014461874961853027, 0.1742398738861084],
[2.1766545821209344e-05, 0.0014046859741210938, 0.1713089942932129],
[2.1931695938110353e-05, 0.0014385533332824706, 0.17559289932250977],
[4.1476978975183824e-05, 0.0024589896202087402, 0.2774052619934082],
[3.978692568265475e-05, 0.002411224842071533, 0.27361512184143066],
[3.982440480646097e-05, 0.002450616359710693, 0.2762136459350586],
[3.8515770876849137e-05, 0.0024203014373779296, 0.27489638328552246],
[3.796811537309126e-05, 0.00243056058883667, 0.2768900394439697],
[3.7687335695539204e-05, 0.002453920841217041, 0.27727627754211426],
[3.636991768552546e-05, 0.002417783737182617, 0.27463674545288086],
[3.648823705212823e-05, 0.002460780143737793, 0.2781810760498047],
[3.5299244573560814e-05, 0.0024231791496276854, 0.27570629119873047],
[3.5325129826863606e-05, 0.0024632978439331054, 0.2792689800262451],
[3.473168513813957e-05, 0.0024625134468078615, 0.2798130512237549],
[3.356645184178506e-05, 0.002422542572021484, 0.27807140350341797],
[3.371276552714999e-05, 0.0024660086631774904, 0.281280517578125],
[3.245975822210312e-05, 0.002419641017913818, 0.2778005599975586],
[3.26241346505972e-05, 0.0024627399444580077, 0.28177309036254883],
[3.212690353393555e-05, 0.002465054988861084, 0.2789885997772217],
[3.148392065247493e-05, 0.002452232837677002, 0.27910804748535156],
[3.1184869654038376e-05, 0.002464628219604492, 0.27913618087768555],
[3.0187420223070226e-05, 0.002425072193145752, 0.2767024040222168],
[3.0258825847080775e-05, 0.002461411952972412, 0.27878713607788086],
[2.9567698357810437e-05, 0.0024424099922180174, 0.27847743034362793],
[2.9453237851460774e-05, 0.002464444637298584, 0.27909064292907715],
[2.8411917490501928e-05, 0.0024144148826599123, 0.2758145332336426],
[2.8636455535888672e-05, 0.0024632954597473143, 0.27903246879577637],
[2.784099578857422e-05, 0.002430927753448486, 0.2778632640838623],
[2.804596173135858e-05, 0.002475755214691162, 0.28269338607788086],
[2.7534311467950994e-05, 0.00246373176574707, 0.2793428897857666],
[2.6732469216371192e-05, 0.002430155277252197, 0.2777373790740967],
[2.6871494100063663e-05, 0.002468059062957764, 0.28216052055358887],
[2.6016086339950563e-05, 0.002426793575286865, 0.28058934211730957],
[2.5616898948763623e-05, 0.0024170708656311036, 0.2796969413757324],
[2.5391840353244688e-05, 0.002424051761627197, 0.28060364723205566],
[2.5260046303990375e-05, 0.002440640926361084, 0.2825767993927002],
[2.4753894124712263e-05, 0.002423455715179443, 0.2802400588989258],
[2.4424272425034468e-05, 0.0024193024635314943, 0.2805345058441162],
[2.4190947066905887e-05, 0.0024231553077697756, 0.2809789180755615],
[2.409170413839406e-05, 0.002440333366394043, 0.2828209400177002],
[2.414017915725708e-05, 0.0024683284759521485, 0.2829020023345947],
[2.3587366168418628e-05, 0.0024424219131469727, 0.282454252243042],
[2.313317192925347e-05, 0.002423856258392334, 0.2792670726776123],
[2.3724928007021057e-05, 0.002505645751953125, 0.2897341251373291],
[2.261016679846722e-05, 0.002421751022338867, 0.2791001796722412],
[2.2453595233219925e-05, 0.0024299144744873045, 0.2821478843688965],
[2.208372379871125e-05, 0.002417871952056885, 0.28142762184143066],
[2.2189667350367496e-05, 0.0024522805213928224, 0.28415942192077637],
[2.1633456150690714e-05, 0.002421116828918457, 0.28116512298583984],
[2.139725635961159e-05, 0.002416536808013916, 0.27909374237060547],
[2.1214339197898397e-05, 0.002420837879180908, 0.28144359588623047],
[2.2677243357957013e-05, 0.002612929344177246, 0.3036656379699707],
[2.2019171714782713e-05, 0.002557480335235596, 0.2973794937133789]
]
)
python_data = np.array([
    [0.0002387103796005249,0.0006598797082901001,9.01147723197937],
[0.00022365880012512206,0.0008685851335525512,11.106118440628052],
[0.00021801133950551352,0.001076094388961792,13.180331945419312],
[0.00021392916440963745,0.001280285859107971,15.225905418395996],
[0.00021358229637145996,0.0014958619832992554,17.397912979125977],
[0.00021005425055821737,0.0016868484735488892,19.302841186523438],
[0.0002098095144544329,0.0018964375734329224,21.40614414215088],
[0.00020817059874534606,0.0020918861865997316,23.365346431732178],
[0.00020755981869167753,0.002295800256729126,25.400399208068848],
[0.00020787601232528685,0.0025081286907196044,27.52853226661682],
[0.00020659196376800537,0.0027019320249557495,29.47439193725586],
[0.0002064779043197632,0.0029076266527175904,31.527666568756104],
[0.00020629437703352708,0.003112311339378357,33.57447385787964],
[0.00020751103503363472,0.0033370547294616698,35.84706473350525],
[0.00020586173216501872,0.003518932485580444,37.64771866798401],
[0.0002055614709854126,0.003720987153053284,39.67592000961304],
[0.00020642429520102107,0.003942117643356323,41.89858031272888],
[0.00020541369120279947,0.004128691267967224,43.75350546836853],
[0.00020591482112282202,0.004346090221405029,45.94392395019531],
[0.00020703518867492676,0.0045753974199295045,48.24669003486633],
[0.00020504196144285655,0.004737956619262696,49.85789394378662],
[0.00020638959407806397,0.0049738689184188846,52.219677448272705],
[0.0002073906784472258,0.005205669355392456,54.550662994384766],
[0.00020780799190203348,0.005424612379074096,56.741135597229004],
[0.00020449754524230957,0.005546933150291443,57.95632982254028],
[0.00020581420751718374,0.005786867833137512,60.349655628204346],
[0.0002051038212246365,0.0059731365919113155,62.22633767127991],
[0.00020478683369500297,0.0061702244281768796,64.20467233657837],
[0.00020486341920392267,0.006377567172050476,66.27745842933655],
[0.00020459582408269245,0.006572646617889404,68.21390962600708],
[0.00020508840237894367,0.006793377804756164,70.43350791931152],
[0.00020573686212301253,0.007019694185256958,72.69663190841675],
[0.00020502450321659897,0.007201675748825073,74.51683354377747],
[0.00020525555400287404,0.007416094732284546,76.6629273891449],
[0.0002054001542500087,0.007625132298469543,78.74842190742493],
[0.00020596563617388406,0.007852475428581238,81.03686547279358],
[0.00020517878596847122,0.008028169322013855,82.78695869445801],
[0.00020599375774985865,0.008265452909469605,85.16409420967102],
[0.0002064468169823671,0.008491739726066589,87.43205547332764],
[0.00020696172058582305,0.008717565441131591,89.69178247451782],
[0.00020623892225870273,0.008894619989395142,91.46094846725464],
[0.00020699531975246612,0.009133046078681945,93.85449147224426],
[0.00020700361506883488,0.009341829657554626,95.93082332611084],
[0.00020719891353086992,0.009556361174583436,98.0767548084259],
[0.00020618683444129096,0.00971778666973114,99.69323825836182],
[0.00020666887967482858,0.009948142671585083,102.00173544883728],
[0.0002072408802965854,0.010182723689079284,104.35649681091309],
[0.00020748129536708195,0.010400817012786865,106.53537702560425],
[0.00020685397800134154,0.010578639769554138,108.3187608718872]
])

In [ ]:
dma_data

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
matplotlib.rcParams['figure.figsize'] = [10, 10]

xpoints = np.arange(1, 101)
ypoints = dma_data[:, 2]

plt.plot(xpoints, ypoints)
plt.show()

In [ ]:

# create data
x = np.arange(1, 100)

# plot lines
plt.plot(x, dma_data[:, 2], label = "FPGA Hardware")
plt.plot(x, python_data[:, 2], label = "Python")
plt.legend()
plt.xlabel("Number of quadratic objects")
plt.ylabel("Time taken per intersection (s)")
plt.grid()

plt.show()

In [ ]:

# create data
x = np.arange(1, 50)

# plot lines
plt.plot(x, dma_data[:, 0], label = "FPGA Hardware")
plt.plot(x, python_data[:, 0], label = "Python")
plt.legend()
plt.xlabel("Number of quadratic objects")
plt.ylabel("Time taken per intersection (s)")
plt.grid()

plt.show()